In [ ]:
import plotnine as p9
import pandas as pd
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

from plotnine_prism import *

In [ ]:
time_df = pd.read_csv("../simulated_data_run_time/run_time.csv")
time_df["train"] = time_df["train"] * 10
#time_df["overall"] = time_df.apply(lambda x: x.prepare_input_for_model + x.train + x.compute_spot_representations, axis=1)

In [ ]:
time_df

In [ ]:
rename_dict = {"prepare_input_for_model": "Grid building",
              "train": "Train\n(100 epochs)",
              "compute_spot_representations": "Inference"}

In [ ]:
to_plot = time_df.melt(["n_points"], var_name="function", value_name="seconds")
to_plot["function"] = to_plot["function"].apply(lambda x: rename_dict[x])
to_plot["function"] = pd.Categorical(to_plot["function"], rename_dict.values())
to_plot["minutes"] = to_plot["seconds"] / 60
to_plot["hours"] = to_plot["minutes"] / 60

p = (p9.ggplot(to_plot, p9.aes("n_points", "minutes", color="function", group="function")) 
 + p9.geom_point(size=2, shape="D") 
 + p9.geom_line(linetype="--", size=1.5, alpha=0.3)
 + p9.theme_bw()
 #+ p9.scale_x_log10()
 #+ p9.scale_y_log10()
 + scale_color_prism(palette = "colors")
 + p9.scale_x_continuous(name='Spots', trans='log10', 
                         breaks=[1_000, 10_000, 100_000, 1_000_000, 10_000_000], 
                         labels=['$10^3$', '$10^4$', '$10^5$', '$10^6$', '$10^7$'])
 + p9.scale_y_continuous(name='Time (min)', trans='log10', 
                         breaks=[0.1, 1, 10, 100, 1000], 
                         labels=['$10^{-1}$', '$10^0$', '$10^1$', '$10^2$', '$10^3$'])
 + p9.guides(color=p9.guide_legend(title="AESTETIK"))
 + p9.theme(figure_size=(6, 4),
            #legend_title = p9.element_blank(),
            text=p9.element_text(size=12),
            strip_text=p9.element_text(size=15),
            legend_text=p9.element_text(size=12))
)
p.save("figures/aestetik_run_time.png", dpi=300)
p